<a href="https://colab.research.google.com/github/VladislavDDP/NN/blob/main/NN_lab8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Лабораторна робота 8
студента групи КН-31 Кучерука Владислава

#### Завдання
1. Знайти в інтернеті матеріал за напрямком дисципліни, що не розглядався на лекціях
2. Навести приклад розв'язання відповідного завдання
3. Підготувати презентацію матеріалу


Постановка задачі: згенерувати словосполучення/речень на основі 

In [1]:
# useful imports

import numpy
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from nltk import *

In [4]:
file = open("text_for_train.txt").read()

In [8]:
download('stopwords')

def tokenize_words(input):
    input = input.lower()

    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(filtered)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
processed_inputs = tokenize_words(file)

In [10]:
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 269878
Total vocab: 43


In [11]:
seq_length = 100
x_data = []
y_data = []

for i in range(0, input_len - seq_length, 1):
    in_seq = processed_inputs[i:i + seq_length]

    out_seq = processed_inputs[i + seq_length]

    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 269778


In [12]:
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

y = np_utils.to_categorical(y_data)

In [13]:
text_generator_model = Sequential([
  LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
  Dropout(0.2),
  LSTM(256,return_sequences=True),
  Dropout(0.2),
  LSTM(128),
  Dropout(0.2),
  Dense(y.shape[1], activation='softmax')
])

text_generator_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [14]:
filepath = "text_generator_model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [15]:
text_generator_model.fit(X, y, epochs=4, batch_size=256, callbacks=desired_callbacks)

Epoch 1/4
1054/1054 [==============================] - ETA: 0s - loss: 2.8465
Epoch 1: loss improved from inf to 2.84651, saving model to text_generator_model_weights_saved.hdf5
1054/1054 [==============================] - 80s 68ms/step - loss: 2.8465
Epoch 2/4
1054/1054 [==============================] - ETA: 0s - loss: 2.5557
Epoch 2: loss improved from 2.84651 to 2.55568, saving model to text_generator_model_weights_saved.hdf5
1054/1054 [==============================] - 72s 68ms/step - loss: 2.5557
Epoch 3/4
1054/1054 [==============================] - ETA: 0s - loss: 2.3767
Epoch 3: loss improved from 2.55568 to 2.37671, saving model to text_generator_model_weights_saved.hdf5
1054/1054 [==============================] - 72s 68ms/step - loss: 2.3767
Epoch 4/4
1054/1054 [==============================] - ETA: 0s - loss: 2.2445
Epoch 4: loss improved from 2.37671 to 2.24449, saving model to text_generator_model_weights_saved.hdf5
1054/1054 [==============================] - 72s 68ms/

In [24]:
num_to_char = dict((i, c) for i, c in enumerate(chars))

start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
" ttered articulate sounds appeared language neither understood understood cottagers made many signs c "


In [26]:
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_len)
    prediction = text_generator_model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = num_to_char[index]

    sys.stdout.write(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]

ontent sear sear sear sear sear sear sear sear sear sear sear sear sear sear sear sear sear sear sea

In [27]:
filepath = "text_generator_model_weights_saved_v2.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

text_generator_model.fit(X, y, epochs=25, batch_size=256, callbacks=desired_callbacks)

Epoch 1/25
1054/1054 [==============================] - ETA: 0s - loss: 2.1409
Epoch 1: loss improved from inf to 2.14092, saving model to text_generator_model_weights_saved_v2.hdf5
1054/1054 [==============================] - 73s 69ms/step - loss: 2.1409
Epoch 2/25
1054/1054 [==============================] - ETA: 0s - loss: 2.0637
Epoch 2: loss improved from 2.14092 to 2.06373, saving model to text_generator_model_weights_saved_v2.hdf5
1054/1054 [==============================] - 72s 69ms/step - loss: 2.0637
Epoch 3/25
1054/1054 [==============================] - ETA: 0s - loss: 2.0020
Epoch 3: loss improved from 2.06373 to 2.00204, saving model to text_generator_model_weights_saved_v2.hdf5
1054/1054 [==============================] - 72s 69ms/step - loss: 2.0020
Epoch 4/25
1054/1054 [==============================] - ETA: 0s - loss: 1.9519
Epoch 4: loss improved from 2.00204 to 1.95188, saving model to text_generator_model_weights_saved_v2.hdf5
1054/1054 [===========================

In [33]:
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_len)
    prediction = text_generator_model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = num_to_char[index]

    sys.stdout.write(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]

 sea strange sea strange sea strange sea strange sea strange sea strange sea strange sea strange sea

## Висновок
На лабораторній роботі було застосовано рекурентну нейронну мережу для генерації текстів. Для аналізу було взято один з корпусів nltk. Результат вийшов не дуже вдалим, оскільки нейромережа часто зациклюється на одному з паттернів.